<h1>Pre-proccessing data into keyword strings</h1>

In [1]:
import pandas as pd
import numpy as np
import csv
from keybert import KeyBERT
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

/Users/theo/opt/anaconda3/envs/fp/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('../train_40k.csv')
# set to cat1 for 6 topics with 2500 entires each
# change call cat1 to Cat2, and replace 2500 with 1000 to create 15 topic model.
df = df[['Text', 'Cat1']]
df = df[df.groupby('Cat1').Cat1.transform('count')>2500].copy()

data = df.values.tolist()

# creates a dictionary of all usable topics 
df_dict = {}
for index, row in df.iterrows():
    df_dict.update({row['Cat1']: 0})

In [3]:
print(len(df_dict))

6


In [4]:
print(df_dict)

{'grocery gourmet food': 0, 'toys games': 0, 'beauty': 0, 'health personal care': 0, 'baby products': 0, 'pet supplies': 0}


In [5]:
kw_model = KeyBERT()

In [6]:
# Write to new CSV
def writeToFile(sen):
    with open('sens13.csv', 'a') as file:
        writer = csv.writer(file)
        row = [sen]
        writer.writerow(row)

In [7]:
# KeyBERT keyword extraction and sentence generation
def processSen(doc, topic):
    keywords = kw_model.extract_keywords(doc, stop_words='english');
    keywordsSen = ''
    keywords.reverse()
    for index, tuple in enumerate(keywords):
        keywordsSen += tuple[0]
        keywordsSen += ' '
    keywordsSen += topic
    writeToFile(keywordsSen)

In [8]:
total = len(df_dict)*2500
total

15000

In [9]:
total = len(df_dict)*2500
count = 0
for sentence in data:
    doc = sentence[0]
    label = sentence[1]
    # Adds - to topic incase topic isnt a single word. This way the entire topic title is included
    topic = sentence[1].replace(" ", "-")
    # Checks the text is longer than 10 words and the count for that topic is less than 2500
    if((len(doc.split()) > 10) and (df_dict[label] < 2500)): 
        # proccesses senteence with keybert and writes to CSV
        # increases count for topic in df_dict
        df_dict[sentence[1]] = df_dict[sentence[1]]+1
        count = count + 1
        processSen(doc, topic)
    else:
        continue
        
    if(count>=total):
        break
print(count)

15000
